# Olá Mundo Concorrente

## Como animar texto na saída padrão

Imagine um **teletipo** ([teletype](http://www.columbia.edu/cu/computinghistory/teletype/index.html)): uma máquina de escrever automatizada que pode trocar mensagens via modem ou comunicação serial com outro teletipo ou com um computador.

<img src="pcworld-sel.jpg" alt="Foto de anúncio mostrando mulher operando minicomputador através de um teletipo" width="30%">

Por isso a tabela ASCII inclui códigos de controle como estes:

|dec|hex |nome           |sigla|Python|
|--:|----|---------------|:---:|------|
|  7|0x07|bell (alarm)   |BEL  |`'\a'`|
|  8|0x08|backspace      |BS   |`'\b'`|
|  9|0x09|horizontal tab |HT   |`'\t'`|
| 10|0x0a|line feed      |LF   |`'\n'`|
| 11|0x0b|vertical tab   |VT   |`'\v'`|
| 12|0x0c|form feed      |FF   |`'\f'`|
| 13|0x0d|carriage return|CR   |`'\r'`|

**Nota:** Nem todos esses códigos funcionam no Jupyter Notebook ou em alguns terminais.

In [1]:
import time

def relogio():
    while True:
        print('\r' + time.strftime('%H:%M:%S'), end='')
        time.sleep(1)

In [2]:
# relogio()

In [3]:
def girar():
    for char in '|/-\\':
        print(f'\r{char}', end=' ')
        time.sleep(1)

In [4]:
girar()

\ 

In [5]:
import itertools

def girar():
    for char in itertools.cycle('|/-\\'):
        print(f'\r{char}', end=' ', flush=True)
        time.sleep(.1)

In [6]:
#girar()

## Exemplo sequencial

In [7]:
from unicodedata import lookup
DOT_CODES = (1, 2, 3, 7, 8, 6, 5, 4)
PREFIX = 'BRAILLE PATTERN DOTS-'
DOTS = ''.join(lookup(f'{PREFIX}{dot}') for dot in DOT_CODES)

In [8]:
def girar(msg):
    for char in itertools.cycle(DOTS):
        print(f'\r{char} {msg}', end=' ', flush=True)
        time.sleep(.1)

def main():
    girar('pensando para sempre...')

In [9]:
#main()

## Exemplos com threads

Duas threads: a thread principal e aquela nós criamos explicitamente.

In [10]:
import itertools
import time
from threading import Thread, Event

def girar(msg: str, calculado: Event) -> None:
    for char in itertools.cycle(r'\|/-'):
        status = f'\r{char} {msg}'
        print(status, end='', flush=True)
        if calculado.wait(0.05):
            break
    blanks = ' ' * len(status)
    print(f'\r{blanks}\r', end='')


def buscar() -> int:
    time.sleep(3)
    return 42


def main():
    pronto = Event()
    fio = Thread(target=girar, args=['buscando...', pronto])
    fio.start()
    res = buscar()
    pronto.set()
    fio.join()
    print(res)


In [11]:
main()

42            


## Exemplos com processos ou corrotinas

As versões com processos (`multiprocessing`) ou corrotinas (`asyncio`) não funcionam aqui por diferentes motivos ligados à
[arquitetura do no Jupyter Notebook](https://docs.jupyter.org/en/latest/projects/architecture/content-architecture.html#the-jupyter-notebook-interface).

<img src="notebook_components.png" width="50%">

**Dica:** Rode no terminal os programas `gira_proc.py` e `gira_async.py`.